# Tutorial 3: Learning algorithms that handle noisy labels

This module will provide examples showing how one would implement a learning algorithm specifically designed for handling noisy labels. 

We will introduce robust loss functions that are ready to be plugged into the empirical risk minimization (ERM) framework. The specific loss functions include [forward/backward loss correction](https://openaccess.thecvf.com/content_cvpr_2017/html/Patrini_Making_Deep_Neural_CVPR_2017_paper.html), loss reweighting [[paper 1](https://proceedings.neurips.cc/paper/2019/hash/9308b0d6e5898366a4a986bc33f3d3e7-Abstract.html), [paper 2](https://ieeexplore.ieee.org/abstract/document/7159100?casa_token=_bObUprbmdIAAAAA:_nx8tU9bE6_6MUw2-6A72c2K_pUtOcEyAm40V70rsaRFRTnPildNJ7qo6FT5mvdOJgP4nwTb8g)], peer loss [[paper 1](http://proceedings.mlr.press/v119/liu20e.html), [paper 2](https://openreview.net/forum?id=WesiCoRVQ15)], [bi-tempered loss](https://proceedings.neurips.cc/paper/2019/hash/8cd7775f9129da8b5bf787a063d8426e-Abstract.html), and label smoothing [[paper 1](http://proceedings.mlr.press/v119/lukasik20a.html?ref=https://githubhelp.com), [paper 2](https://proceedings.mlr.press/v162/wei22b.html)]. We will also discuss other approaches, including [Co-Teaching](https://proceedings.neurips.cc/paper/8072-co-teaching-robust-training-o), and [DivideMix](https://openreview.net/forum?id=HJgExaVtwr), which employ two neural networks to supervise each other.


### The role of noise transition matrix

For the K-class classification task, we have

$
    \underbrace{\begin{bmatrix}
    \mathbb{P}(\widetilde{Y}=1)\\
    \mathbb{P}(\widetilde{Y}=2)\\
    ...\\
    \mathbb{P}(\widetilde{Y}=K)
    \end{bmatrix}}_{\text{Observation}}=\underbrace{\begin{bmatrix}
    \mathbb{P}(\widetilde{Y}=1|Y=1) & \mathbb{P}(\widetilde{Y}=1|Y=2) & ... & \mathbb{P}(\widetilde{Y}=1|Y=K)\\
    \mathbb{P}(\widetilde{Y}=2|Y=1) & \mathbb{P}(\widetilde{Y}=2|Y=2) & ... & \mathbb{P}(\widetilde{Y}=2|Y=K)\\
    ...& ... & ... & ...\\ \mathbb{P}(\widetilde{Y}=K|Y=1) & \mathbb{P}(\widetilde{Y}=1|Y=2) & ... & \mathbb{P}(\widetilde{Y}=1|Y=K)
    \end{bmatrix}}_{\text{Class-dependent noise transition matrix }T^{\intercal}}\cdot \underbrace{\begin{bmatrix}
    \mathbb{P}(Y=1)\\
    \mathbb{P}(Y=2)\\
    ...\\
   \mathbb{P}(Y=K)
    \end{bmatrix}}_{\text{Hidden truth}}.
$

A simplified binary case can be defined as (Class $+1$, Class $-1$):

$
     \begin{bmatrix}
    \mathbb{P}(\widetilde{Y}=+1)\\
    \mathbb{P}(\widetilde{Y}=-1)
    \end{bmatrix}  =\underbrace{\begin{bmatrix}
    1-e_+ & e_-  \\
    e_+ & 1-e_-  
    \end{bmatrix}}_{T_{2\times 2}^{\intercal}}\cdot \underbrace{\begin{bmatrix}
    \mathbb{P}(Y=+1)\\
    \mathbb{P}(Y=-1)
    \end{bmatrix}}_{\text{Hidden truth}},
$

where $e_+:=\mathbb{P}(\widetilde{Y}=-1|Y=+1), e_-:=\mathbb{P}(\widetilde{Y}=+1|Y=-1)$.

## Backward Loss Correction
 
**Loss correction**

For the noisy data sample $(x, \tilde{y})$, suppose the clean label of $x$ is $y$:
$$\ell_{\text{LC}}(h(x), \tilde{y}):=\frac{(1-e_{-\tilde{y}})}{1-e_+-e_-}\cdot {\color{red}{\ell(h(x),\tilde{y})}}-\frac{e_{\tilde{y}}}{1-e_+-e_-}\cdot {\color{blue}{\ell(h(x),-\tilde{y})}}.$$

**<font color='red'> Remark: </font>**
$-\tilde{y}$ simply encodes the class differs from $\tilde{y}$.

**Motivation of loss correction** (i.e., for class $y=+1$)

$$\mathbb{E}_{\tilde{y}|y=+1}[\ell_{\text{LC}}(h(x), \tilde{y})]=\underbrace{(1-e_+)}_{\mathbb{P}(\tilde{y}=+1|y=+1)}\cdot \ell_{\text{LC}}(h(x), \tilde{y}=+1)+\underbrace{e_+}_{\mathbb{P}(\tilde{y}=-1|y=+1)}\cdot \ell_{\text{LC}}(h(x), \tilde{y}=-1).$$

Substitute $\ell_{\text{LC}}(h(x), \tilde{y}=+1)$ and $\ell_{\text{LC}}(h(x), \tilde{y}=-1)$, we have

$$\mathbb{E}_{\tilde{y}|y=+1}[\ell_{\text{LC}}(h(x), \tilde{y})]=\underbrace{\ell(h(x), y=+1)}_{{\color{red}{\text{Loss on the clean data!}}}}.$$